## Experiments

Let's start playing around with our data

In [36]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import nltk
nltk.download('punkt')
from sklearn.feature_extraction import DictVectorizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /home/jonat/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Read in training data
This dataset contains 100 annotated terms of service contracts, each row represents a sentence, which carries on it a label. The label corresponds to a different type of potential unfairness, as defined by the authors of CLAUDETTE, the previous paper from which this dataset came from. 

In [37]:
df = pd.read_csv('../data/dataset.csv')
df.head()


,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


get the vocabulary for the dataset

In [ ]:
# from datasets import load_dataset
# #load in the pile of law dataset
# pol = load_dataset("pile-of-law/pile-of-law", 'tos',split="train + validation")

Now let's load in a pretrained huggingface BERT model  (https://huggingface.co/pile-of-law/legalbert-large-1.7M-2) to get the word embeddings of each sentence

In [74]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model = BertModel.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model.to(device)
text = "This is a test"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input.to(device))

In [78]:
display(output.last_hidden_state[0].shape)
output.last_hidden_state[0]

torch.Size([6, 1024])

tensor([[-0.9083, -1.3334,  0.8472,  ..., -1.1469,  0.9403, -0.3372],
        [ 0.9027, -0.1450, -0.8254,  ..., -0.9643,  0.4075, -0.7058],
        [-0.5211, -1.0929,  0.6936,  ..., -0.6173,  0.5492,  1.2246],
        [ 0.0312, -0.6487,  0.2369,  ...,  0.4785, -1.4233,  0.0361],
        [-1.9127, -0.3721, -0.0364,  ...,  0.4318,  1.1359,  0.1359],
        [-1.6205, -1.0455,  0.0540,  ..., -1.1866,  0.5167, -0.3113]],
       device='cuda:0', grad_fn=<SelectBackward0>)

The pre trained model gives encodings of shape 12 x 1024 for a sentence, this will be useful to know when we use it as part of our model.